In [2]:
# Install dependencies first: 
!pip install langchain cohere numexpr

In [6]:
# LangChain Agent with Cohere API - Fixed & Tested
# !pip install langchain cohere numexpr

import os
from langchain.agents import AgentType, initialize_agent, Tool
from langchain.llms import Cohere
import re
import numexpr

# ========================
# SETUP & INITIALIZATION
# ========================
os.environ["COHERE_API_KEY"] = "your_api_key"  # Replace with your key

# Initialize Cohere LLM - optimized for math reasoning
llm = Cohere(
    model="command",  # Use base command model
    temperature=0.0,  # Zero temperature for deterministic math
    max_tokens=150
)

# ========================
# CUSTOM MATH TOOL (FIXED)
# ========================
def safe_math_evaluator(expression: str) -> str:
    """Safely evaluates mathematical expressions with Cohere-specific fixes"""
    # Clean and convert expression
    expression = expression.strip()
    expression = expression.replace(' of ', ' * ').replace('percent', '*0.01')
    expression = re.sub(r'[^0-9\.\+\-\*\/\(\) ]', '', expression)
    
    try:
        result = numexpr.evaluate(expression)
        return f"{result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Create tool with EXACT name the agent should use
math_tool = Tool(
    name="MathCalculator",  # MUST match what agent calls
    func=safe_math_evaluator,
    description=(
        "Use ONLY for math calculations. Input MUST be a pure numerical expression. "
        "Examples: '250 * 0.18', '45 - 15'. NEVER include words in the input."
    )
)

tools = [math_tool]

# ========================
# AGENT CREATION (FIXED)
# ========================
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,
    early_stopping_method="generate"
)

# ========================
# EXECUTION & OUTPUT
# ========================
query = """
First calculate 18% of 250, 
then subtract 15 from that result, 
and finally add 100 to the total.
"""

print("🧠 USER QUERY:", query)
response = agent.run(
    "Calculate step-by-step: " + query + " Use ONLY MathCalculator for computations."
)
print(f"\n🌟 FINAL RESULT: {response}")

🧠 USER QUERY: 
First calculate 18% of 250, 
then subtract 15 from that result, 
and finally add 100 to the total.



> Entering new AgentExecutor chain...


unknown field: parameter 'model' is not a valid field


 This seems like a fairly straightforward math problem. I will use the MathCalculator tool to compute each step sequentially.

Action: MathCalculator
Action Input: 250 * 0.18
Observation: 45.0
Thought:

unknown field: parameter 'model' is not a valid field


 Thought: Great, I now have the first intermediate result, which is 45. I will proceed with the next step.
Action: MathCalculator
Action Input: 45 - 15
Observation: 30
Thought:

unknown field: parameter 'model' is not a valid field


 Thought: Okay, I have computed the second intermediate result, which is 30. I need to add 100 to this result now.
Action: MathCalculator
Action Input: 30 + 100
Observation: 130
Thought:

unknown field: parameter 'model' is not a valid field


 Thought: I believe the final answer is 130, based on the steps and calculations performed using the MathCalculator tool.
Final Answer: 130

> Finished chain.

🌟 FINAL RESULT: 130


# What's Happening Behind the Scenes

## 1. Input Handling
- Your natural language query is received  
- Cohere LLM parses it into executable steps:  
  → Calculate 18% of 250  
  → Subtract 15 from result  
  → Add 100 to new result  

## 2. Reasoning Process
- Uses chain-of-thought prompting to break down the problem  
- Delegates each calculation to the MathCalculator tool  
- Maintains context between steps (output from step 1 becomes input for step 2)  

## 3. Tool Execution
- Converts natural language to math operations:  
  "18% of 250" → `250 * 0.18`  
- Executes calculations safely using `numexpr`  
- Returns precise numerical results:  
  `45.0 → 30.0 → 130.0`  

## 4. Response Generation
- Combines tool outputs with natural language  
- Formats final explanation:  
  "18% of 250 (45), subtract 15 (30), add 100 = 130"  

---

# Simplified Execution Flow

```mermaid
graph LR
    A[User Query] --> B[Cohere: Plan Steps]
    B --> C[Math Tool: Calculate]
    C --> D[Cohere: Explain]
    D --> E[Final Answer]results.